# TCC Focos de Queimadas no Cerrado
## Entendimento inicial da base INMET + BDQueimadas

Este caderno implementa a etapa 1 da análise: inspeção e auditoria de qualidade dos arquivos ano a ano no padrão `data/dataset/inmet_bdq_{ANO}_cerrado.csv`. O foco aqui é medir:

1. Proporção de HAS_FOCO por ano.
2. Ocorrência de valores sentinela em qualquer coluna numérica: -999, -9999, 999, 9999.
3. Entre as linhas com foco, quantas contêm sentinelas negativos (-999 ou -9999).

Observação de parsing: os CSVs usam vírgula como separador decimal em diversas colunas. Usaremos `decimal=','` no carregamento e coerção numérica laxa para auditoria.


## Sumário

1. [Configuração e imports](#configuracao-e-imports)
2. [Parâmetros do projeto e diretórios](#parametros-do-projeto-e-diretorios)
3. [Funções de descoberta e leitura](#funcoes-de-descoberta-e-leitura)
4. [Coerção numérica e sentinelas](#coercao-numerica-e-sentinelas)
5. [Métricas por ano](#metricas-por-ano)
6. [Pipeline de auditoria](#pipeline-de-auditoria)
7. [Execução e persistência do resumo](#execucao-e-persistencia-do-resumo)
8. [Visualizações rápidas](#visualizacoes-rapidas)
9. [Sanity check com 2003](#sanity-check-com-2003)
10. [Próximos passos](#proximos-passos)


## 1. Configuração e imports

In [1]:
from __future__ import annotations

from pathlib import Path
from dataclasses import dataclass
from typing import Iterable, Dict, List

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

## 2. Parâmetros do projeto e diretórios

In [2]:
# Raiz do projeto e subpastas relevantes
PROJECT_ROOT = Path(".").resolve()
DATASET_DIR = PROJECT_ROOT / "data" / "dataset"
REPORTS_DIR = PROJECT_ROOT / "reports" / "eda"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Padrão de nome dos arquivos ano a ano
FILENAME_PATTERN = "inmet_bdq_*_cerrado.csv"

# Valida diretórios
assert DATASET_DIR.exists(), f"Diretório não encontrado: {DATASET_DIR}"
print("Dataset dir:", DATASET_DIR)
print("Reports dir:", REPORTS_DIR)

AssertionError: Diretório não encontrado: D:\Projetos\TCC\src\data\dataset

## 3. Funções de descoberta e leitura

In [9]:
def load_minimal_has_foco(path: Path) -> pd.Series:
    """
    Lê apenas HAS_FOCO; retorna Series int64 {0,1}.
    """
    df = pd.read_csv(path, dtype=str, usecols=["HAS_FOCO"], low_memory=False, encoding="utf-8")
    return pd.to_numeric(df["HAS_FOCO"], errors="coerce").fillna(0).astype("int64")

def summarize_file(year: int, path: Path) -> dict:
    s = load_minimal_has_foco(path)
    n_rows = int(s.shape[0])
    n_focos = int((s == 1).sum())
    prop = float(n_focos / n_rows) if n_rows > 0 else 0.0
    return {"year": year, "file": str(path), "rows": n_rows, "focos": n_focos, "prop_foco": prop}

In [10]:
files = find_dataset_files(BIOME, YEARS)
if not files:
    raise FileNotFoundError(
        f"Nenhum arquivo encontrado em {DATASET_DIR} com padrão inmet_bdq_{{YYYY}}_{BIOME}.csv.\n"
        f"Dica: verifique a célula 0 para a lista de arquivos detectados."
    )

summary_rows = [summarize_file(y, p) for y, p in files]
summary_df = pd.DataFrame(summary_rows).sort_values("year").reset_index(drop=True)

summary_df_style = (
    summary_df
    .assign(prop_foco_pct=(summary_df["prop_foco"] * 100.0))
    .drop(columns=["prop_foco"])
    .rename(columns={"prop_foco_pct": "prop_foco(%)"})
)
summary_df_style

KeyboardInterrupt: 

In [ ]:
total_rows = int(summary_df["rows"].sum())
total_focos = int(summary_df["focos"].sum())
total_prop  = float(total_focos / total_rows) if total_rows > 0 else 0.0

totais = pd.DataFrame([{
    "anos_cobertos": (
        f"{summary_df['year'].min()}–{summary_df['year'].max()}"
        if len(summary_df) > 1 else f"{summary_df['year'].iloc[0]}"
    ),
    "arquivos": len(summary_df),
    "linhas_total": total_rows,
    "focos_total": total_focos,
    "prop_foco_total(%)": total_prop * 100.0,
}])
totais

,anos_cobertos,arquivos,linhas_total,focos_total,prop_foco_total(%)
0,2003–2024,20,45135924,151544,0.33575


In [ ]:
BIOME = "cerrado"
YEARS = [2024]
files_2003 = find_dataset_files(BIOME, YEARS)
if not files_2003:
    raise FileNotFoundError("Arquivo 2003 não encontrado (veja a lista na Célula 0).")

y, p = files_2003[0]
one = summarize_file(y, p)
pd.DataFrame([one]).assign(prop_foco_pct=lambda d: d["prop_foco"] * 100).drop(columns=["prop_foco"])

,year,file,rows,focos,prop_foco_pct
0,2024,D:\Projetos\TCC\data\dataset\inmet_bdq_2024_ce...,3161880,12740,0.402925
